In [181]:
import numpy as np
import wandb
import pandas as pd
from jaxlib.xla_client import tracebacks
from tqdm import tqdm

In [182]:
api = wandb.Api(timeout=600)

In [183]:
runs = api.runs("inchangbaek4907/scenario-llama")
len(runs)

190

In [184]:
runs = list(runs)
runs[:5]

[<Run inchangbaek4907/scenario-llama/pe-cot_it-6_fit-hr_exp-llama33_t-sce_chr-1_1_s-1 (finished)>,
 <Run inchangbaek4907/scenario-llama/pe-got_it-6_fit-hr_exp-llama33_t-sce_chr-2_1_s-1 (finished)>,
 <Run inchangbaek4907/scenario-llama/pe-tot_it-6_fit-hr_exp-llama33_t-sce_chr-2_1_s-1 (finished)>,
 <Run inchangbaek4907/scenario-llama/pe-got_it-6_fit-hr_exp-llama33_t-sce_chr-1_1_s-1 (finished)>,
 <Run inchangbaek4907/scenario-llama/pe-tot_it-6_fit-hr_exp-llama33_t-sce_chr-1_1_s-1 (running)>]

In [185]:

class RunInfo:
    config: dict
    name: str
    run_object: wandb.apis.public.Run
    
    def __init__(self, config, name, run_object):
        self.config = config
        self.name = name
        self.run_object = run_object

    def __str__(self):
        showing_config = dict()
        showing_keys = ['gpt_model']
        
        for key in showing_keys:
            if key in self.config:
                showing_config[key] = self.config[key]
        
        return f"RunInfo(\n\tname={self.name},\n\tconfig={showing_config}\n)"

In [186]:
run_infos = list()

for run in tqdm(runs):
        
    try:
        if run.state != 'running' and ('Evaluation/llm_iteration' not in run.summary.keys()):
            run_infos.append(RunInfo(run.config, run.name, run))
        elif run.state != 'running' and run.summary['Evaluation/llm_iteration'] < 6:
            run_infos.append(RunInfo(run.config, run.name, run))
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(run.id, e)
        continue
len(run_infos)

100%|██████████| 190/190 [00:00<00:00, 107343.45it/s]


6

In [187]:
configs = list()
for run_info in tqdm(run_infos[:]):
    configs.append(run_info.config)
    
df = pd.DataFrame(configs)
df

100%|██████████| 6/6 [00:00<00:00, 35444.82it/s]


,randomize_start_pos,feedback_input_type,random_agent,NUM_UPDATES,empty_start,feedback_type,timestep_chunk_size,n_eps,hidden_dims,bypass_reward_path,...,NUM_MINIBATCHES,eval_seed,total_timesteps,available_tiles,n_generation_trials,max_board_scans,DEBUG,n_codefix_trials,eval_randomize_map_shape,n_eval_maps
0,False,array,False,None,False,default,-1,2,"[64, 256]",None,...,4,0,50000000,[],5,3,True,5,False,6
1,False,array,False,None,False,default,-1,2,"[64, 256]",None,...,4,0,50000000,[],5,3,True,5,False,6
2,False,array,False,None,False,default,-1,2,"[64, 256]",None,...,4,0,50000000,[],5,3,True,5,False,6
3,False,array,False,None,False,default,-1,2,"[64, 256]",None,...,4,0,50000000,[],5,3,True,5,False,6
4,False,array,False,None,False,default,-1,2,"[64, 256]",None,...,4,0,50000000,[],5,3,True,5,False,6
5,False,array,False,None,False,default,-1,2,"[64, 256]",None,...,4,0,50000000,[],5,3,True,5,False,6


In [188]:
commands = list()
ids = list()

def get_config(exp_group, target_character):
    return df[(df['exp_group'] == exp_group) & (df['target_character'] == target_character)].iloc[0]

for index, row in df.iterrows():
    
    config = get_config(row['exp_group'], row['target_character'])
    
    seed = row['seed']
    exp_group = row['exp_group']
    target_character = row['target_character']
    
    total_iterations = config['total_iterations']
    gpt_model = config['gpt_model']
    exp_name = config['exp_name']
    n_self_alignment = config['n_self_alignment']
    evaluator = config['evaluator']
    pe = config['pe']
    
    n_self_alignment = int(n_self_alignment)
    

    command = f"bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe={pe} exp_name={exp_name} gpt_model={gpt_model} total_iterations={total_iterations} seed={seed} evaluator={evaluator} target_character={target_character} n_self_alignment={n_self_alignment} n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow"
    commands.append(command)

commands

['bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=tot exp_name=llama33 gpt_model=llama-33 total_iterations=6 seed=2 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=got exp_name=llama33 gpt_model=llama-33 total_iterations=6 seed=2 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=tot exp_name=llama32 gpt_model=llama-32 total_iterations=6 seed=2 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=got exp_name=llama32 gpt_model=llama-32 tota

In [189]:
len(commands)

6

In [190]:
commands_block = " \n".join(commands)
from IPython.display import display, Markdown
display(Markdown("```bash\n" + commands_block + ""))

```bash
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=tot exp_name=llama33 gpt_model=llama-33 total_iterations=6 seed=2 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=got exp_name=llama33 gpt_model=llama-33 total_iterations=6 seed=2 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=tot exp_name=llama32 gpt_model=llama-32 total_iterations=6 seed=2 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=got exp_name=llama32 gpt_model=llama-32 total_iterations=6 seed=1 evaluator=hr target_character=2 n_self_alignment=0 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=cot exp_name=llama32sa gpt_model=llama-32 total_iterations=6 seed=6 evaluator=hr target_character=2 n_self_alignment=5 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario-llama task=scenario overwrite=True pe=got exp_name=llama32sa gpt_model=llama-32 total_iterations=6 seed=5 evaluator=hr target_character=1 n_self_alignment=5 n_codegen_trials=5 n_codefix_trials=5 wandb_resume=allow

In [191]:
ids

[]